In [ ]:
!pip install pyngrok

In [ ]:
from google.colab import drive

drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
from flask import Flask
from pyngrok import ngrok

In [ ]:
ngrok.set_auth_token("2pMZTEk1QEdSzrIdRPPnEBkH62n_2LkmJ6emMAjy4yjdrb2KL")
public_url = ngrok.connect(5000).public_url

In [ ]:
print(public_url)

https://1708-34-32-255-55.ngrok-free.app


In [ ]:
port_no = 5000

In [ ]:
!pip install pytz


In [ ]:
pip install openpyxl


In [ ]:
from flask import Flask, request, jsonify, render_template, send_from_directory, Response, redirect, url_for, session, send_file
from openpyxl import Workbook


In [ ]:
from openpyxl import Workbook
from openpyxl.styles import Font, PatternFill, Border, Side, Alignment
from flask import send_file


In [ ]:
import torch
import cv2
from flask import Flask, request, jsonify, render_template, send_from_directory, Response, redirect, url_for, session
from pyngrok import ngrok
import threading
import os
import sqlite3
from datetime import datetime
import pytz
from werkzeug.security import generate_password_hash, check_password_hash
import csv


app = Flask(__name__)
app.secret_key = "gizli_anahtarınız"


ngrok.set_auth_token("2pMZTEk1QEdSzrIdRPPnEBkH62n_2LkmJ6emMAjy4yjdrb2KL")
port_no = 5000
public_url = ngrok.connect(port_no).public_url

UPLOAD_FOLDER = 'static/uploads'
os.makedirs(UPLOAD_FOLDER, exist_ok=True)
app.config['UPLOAD_FOLDER'] = UPLOAD_FOLDER


DATABASE = "gecmis.db"

def init_db():
    """Initialize the database and create tables if they don't exist."""
    conn = sqlite3.connect(DATABASE)
    cursor = conn.cursor()

    cursor.execute('''
        CREATE TABLE IF NOT EXISTS patients (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            name TEXT NOT NULL,
            patient_id TEXT NOT NULL UNIQUE,
            email TEXT,
            phone TEXT,
            user_id INTEGER NOT NULL,
            FOREIGN KEY(user_id) REFERENCES users(id)
        )
    ''')

    # Add user_id field to history table
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS history (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            filename TEXT NOT NULL,
            label TEXT NOT NULL,
            date_uploaded TEXT NOT NULL,
            patient_id INTEGER NOT NULL,
            user_id INTEGER NOT NULL,
            FOREIGN KEY(patient_id) REFERENCES patients(id),
            FOREIGN KEY(user_id) REFERENCES users(id)
        )
    ''')

    # Create users table if it doesn't exist
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS users (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            username TEXT NOT NULL UNIQUE,
            email TEXT NOT NULL UNIQUE,
            password TEXT NOT NULL
        )
    ''')

    conn.commit()
    conn.close()

init_db()

print("Veritabanı başarıyla başlatıldı!")

model_path = "/content/drive/MyDrive/DentalRadiographsDataset/best_resnet18_full_model.pth"
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = torch.load(model_path, map_location=device)
model = model.to(device)
print("Model başarıyla yüklendi!")

TURKEY_TIMEZONE = pytz.timezone("Europe/Istanbul")

def classify_image(model, image_path):
    model.eval()


    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    if image is None:
        raise ValueError(f"Görüntü bulunamadı: {image_path}")

    image = cv2.cvtColor(image, cv2.COLOR_GRAY2RGB)


    image = cv2.resize(image, (224, 224))

    image = (image / 127.5) - 1

    image_tensor = torch.tensor(image).permute(2, 0, 1).unsqueeze(0).float().to(device)

    with torch.no_grad():
        output = model(image_tensor)
        prediction = (output > 0.5).float().item()


    label = "Çürük tespit edildi." if prediction == 1 else "Bu dişler sağlıklı."
    return label

@app.route("/")
def home():
    if "user_id" in session:
        return render_template("index.html", username=session["username"])
    return redirect(url_for("login"))

@app.route("/signup", methods=["GET", "POST"])
def signup():
    if request.method == "POST":
        username = request.form["username"]
        email = request.form["email"]
        password = request.form["password"]


        hashed_password = generate_password_hash(password)


        try:
            conn = sqlite3.connect(DATABASE)
            cursor = conn.cursor()
            cursor.execute('''
                INSERT INTO users (username, email, password)
                VALUES (?, ?, ?)
            ''', (username, email, hashed_password))
            conn.commit()
            conn.close()
            return redirect(url_for("login"))
        except sqlite3.IntegrityError:
            return render_template("signup.html", error="Kullanıcı adı veya e-posta zaten mevcut.")
    return render_template("signup.html")

@app.route("/login", methods=["GET", "POST"])
def login():
    if request.method == "POST":
        email = request.form["email"]
        password = request.form["password"]

        conn = sqlite3.connect(DATABASE)
        cursor = conn.cursor()
        cursor.execute("SELECT id, username, password FROM users WHERE email = ?", (email,))
        user = cursor.fetchone()
        conn.close()

        if user and check_password_hash(user[2], password):
            session["user_id"] = user[0]
            session["username"] = user[1]
            return redirect(url_for("home"))
        return render_template("login.html", error="Geçersiz e-posta veya şifre.")
    return render_template("login.html")

@app.route("/logout")
def logout():
    """Log the user out by clearing the session and redirecting to the login page."""
    session.clear()
    return redirect(url_for("login"))


@app.route("/upload", methods=["GET", "POST"])
def upload_file():
    if "user_id" not in session:
        return redirect(url_for("login"))

    conn = sqlite3.connect(DATABASE)
    cursor = conn.cursor()

    if request.method == "POST":
        patient_id = request.form.get("patient_id")
        if not patient_id:
            conn.close()
            return render_template("upload.html", error="Lütfen hasta ID'sini girin.")


        cursor.execute("SELECT id FROM patients WHERE patient_id = ? AND user_id = ?", (patient_id, session["user_id"]))
        patient_record = cursor.fetchone()

        if not patient_record:
            conn.close()
            return render_template("upload.html", error="Girilen hasta ID'si bulunamadı veya size ait değil.")

        if "file" not in request.files:
            conn.close()
            return render_template("upload.html", error="Dosya yüklenmedi.")

        file = request.files["file"]
        if file.filename == "":
            conn.close()
            return render_template("upload.html", error="Dosya adı boş olamaz.")

        file_path = os.path.join(app.config['UPLOAD_FOLDER'], file.filename)
        file.save(file_path)

        try:
            label = classify_image(model, file_path)


            current_time = datetime.now(TURKEY_TIMEZONE).strftime('%Y-%m-%d %H:%M:%S')
            cursor.execute('''
                INSERT INTO history (filename, label, date_uploaded, patient_id, user_id)
                VALUES (?, ?, ?, ?, ?)
            ''', (file.filename, label, current_time, patient_record[0], session["user_id"]))
            conn.commit()


            image_url = f"/static/uploads/{file.filename}"

            conn.close()
            return render_template("upload.html", image_url=image_url, label=label)

        except Exception as e:
            conn.close()
            return render_template("upload.html", error=str(e))


    cursor.execute("SELECT id, patient_id FROM patients WHERE user_id = ?", (session["user_id"],))
    patients = cursor.fetchall()
    conn.close()

    return render_template("upload.html", patients=patients)




@app.route("/dashboard")
def dashboard():
    if "user_id" not in session:
        return redirect(url_for("login"))

    conn = sqlite3.connect(DATABASE)
    cursor = conn.cursor()
    cursor.execute('''
        SELECT h.filename, h.label, h.date_uploaded, p.name
        FROM history h
        LEFT JOIN patients p ON h.patient_id = p.id
        WHERE h.user_id = ?
        ORDER BY h.id DESC
    ''', (session["user_id"],))
    rows = cursor.fetchall()
    conn.close()

    return render_template("dashboard.html", rows=rows, username=session["username"])





@app.route("/patients", methods=["GET", "POST"])
def manage_patients():
    """Manage patient records: Add, view, edit, and delete patients."""
    if "user_id" not in session:
        return redirect(url_for("login"))

    conn = sqlite3.connect(DATABASE)
    cursor = conn.cursor()

    if request.method == "POST":
        # Add a new patient
        name = request.form["name"]
        patient_id = request.form["patient_id"]
        email = request.form.get("email")
        phone = request.form.get("phone")
        user_id = session["user_id"]
        try:
            cursor.execute('''
                INSERT INTO patients (name, patient_id, email, phone, user_id)
                VALUES (?, ?, ?, ?, ?)
            ''', (name, patient_id, email, phone, user_id))
            conn.commit()
        except sqlite3.IntegrityError:
            return render_template("patients.html", error="Patient ID already exists.", patients=fetch_patients())


    patients = fetch_patients(user_id=session["user_id"])
    conn.close()

    return render_template("patients.html", patients=patients)

    return render_template("patients.html", patients=fetch_patients())


def fetch_patients(user_id):
    """Fetch all patients for the logged-in user."""
    conn = sqlite3.connect(DATABASE)
    cursor = conn.cursor()
    cursor.execute("SELECT id, name, patient_id, email, phone FROM patients WHERE user_id = ?", (user_id,))
    patients = cursor.fetchall()
    conn.close()
    return patients





@app.route("/patient/<int:patient_id>")
def patient_history(patient_id):
    """Display the classification history for a specific patient."""
    conn = sqlite3.connect(DATABASE)
    cursor = conn.cursor()

    cursor.execute("SELECT name, patient_id, email, phone FROM patients WHERE id = ?", (patient_id,))
    patient = cursor.fetchone()

    if not patient:
        conn.close()
        return "Bu hasta bulunamadı.", 404

    cursor.execute('''
        SELECT filename, label, date_uploaded
        FROM history
        WHERE patient_id = ?
        ORDER BY date_uploaded DESC
    ''', (patient_id,))
    history = cursor.fetchall()

    conn.close()
    return render_template("patient_history.html", patient=patient, history=history)





@app.route("/download/excel")
def download_history_excel():
    """Generate and download the classification history as an Excel file with improved formatting."""
    conn = sqlite3.connect(DATABASE)
    cursor = conn.cursor()

    cursor.execute('''
        SELECT h.filename, h.label, h.date_uploaded, p.name
        FROM history h
        LEFT JOIN patients p ON h.patient_id = p.id
        ORDER BY h.id DESC
    ''')
    rows = cursor.fetchall()
    conn.close()

    wb = Workbook()
    ws = wb.active
    ws.title = "Classification History"

    header_font = Font(bold=True, color="FFFFFF")
    header_fill = PatternFill(start_color="007bff", end_color="007bff", fill_type="solid")
    header_alignment = Alignment(horizontal="center", vertical="center")
    border_style = Border(left=Side(style="thin"), right=Side(style="thin"), top=Side(style="thin"), bottom=Side(style="thin"))


    headers = ["Dosya Adı", "Etiket", "Yükleme Tarihi", "Hasta Adı"]
    for col_num, header in enumerate(headers, 1):
        cell = ws.cell(row=1, column=col_num, value=header)
        cell.font = header_font
        cell.fill = header_fill
        cell.alignment = header_alignment
        cell.border = border_style

    row_fill_even = PatternFill(start_color="f4f4f9", end_color="f4f4f9", fill_type="solid")
    row_fill_odd = PatternFill(start_color="FFFFFF", end_color="FFFFFF", fill_type="solid")
    for row_num, row in enumerate(rows, 2):
        for col_num, cell_value in enumerate(row, 1):
            cell = ws.cell(row=row_num, column=col_num, value=cell_value)
            cell.border = border_style
            cell.alignment = Alignment(horizontal="center", vertical="center")

            cell.fill = row_fill_even if row_num % 2 == 0 else row_fill_odd


    for column in ws.columns:
        max_length = 0
        column_letter = column[0].column_letter
        for cell in column:
            if cell.value:
                max_length = max(max_length, len(str(cell.value)))
        ws.column_dimensions[column_letter].width = max_length + 5


    excel_file_path = "classification_history.xlsx"
    wb.save(excel_file_path)

    return send_file(
        excel_file_path,
        as_attachment=True,
        download_name="classification_history.xlsx",
        mimetype="application/vnd.openxmlformats-officedocument.spreadsheetml.sheet"
    )



@app.route("/static/uploads/<filename>")
def serve_uploaded_file(filename):
    return send_from_directory(app.config['UPLOAD_FOLDER'], filename)

def run_flask():
    print(f"Küresel bağlantıya erişmek için tıklayın: {public_url}")
    app.run(port=port_no, use_reloader=False)

thread = threading.Thread(target=run_flask)
thread.start()


Veritabanı başarıyla başlatıldı!


<ipython-input-11-cd7b6ede5014>:79: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model = torch.load(model_path, map_location=device)


Model başarıyla yüklendi!
Küresel bağlantıya erişmek için tıklayın: https://47bc-34-32-255-55.ngrok-free.app
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


In [ ]:
import os
os.makedirs("templates", exist_ok=True)


In [ ]:
!ls


drive  gecmis.db  sample_data  static  templates


In [ ]:
%%writefile templates/index.html
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Diş Radyografisi Sınıflandırıcısı</title>
    <link rel="stylesheet" href="/static/style.css">
</head>
<body>


<div class="navbar">
    <a href="/">Ana Sayfa</a>
    <a href="/dashboard">Kontrol Paneli</a>
    <a href="/patients">Hasta Yönetimi</a>
    <a href="/upload">Dosya Yükleme</a>
    <a href="/signup">Hesap Oluştur</a>
    <a href="/logout" class="logout">Çıkış Yap</a>
</div>

    <div class="container">
        <h1>Çocuklar İçin Diş Radyografisi Sınıflandırıcısı</h1>
        <p>
            Bu platform, diş hekimliği profesyonelleri için özel olarak tasarlanmıştır. Çocukların diş radyografilerini analiz ederek çürük veya sağlıklı sınıflandırmalarını yapar.
        </p>
        <p>
            Derin öğrenme teknolojisi kullanılarak teşhis doğruluğunu artırmayı ve hasta bakımını iyileştirmeyi hedefliyoruz.
        </p>
        <a href="/upload" class="download-button">Başlamak için Dosya Yükle</a>
    </div>


    <div class="about-section">
        <h2>Hakkımızda</h2>
        <p>
            <strong>Çocuklar İçin Diş Radyografisi Sınıflandırıcısı</strong>, diş hekimlerini AI destekli araçlarla güçlendirmeyi ve daha doğru teşhisler yapmalarını sağlamayı amaçlar.
        </p>
        <p>
            Amacımız, kullanıcı dostu bir arayüzle verimli bir çözüm sunarak hasta bakımını en üst düzeye çıkarmaktır.
        </p>
    </div>

    <footer>
        <p>&copy; 2025 Çocuklar İçin Diş Radyografisi Sınıflandırıcısı | Developed by Walid Mohmmed</p>
    </footer>

</body>
</html>



Writing templates/index.html


In [ ]:
%%writefile templates/patient_history.html
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Patient History</title>
    <link rel="stylesheet" href="/static/style.css">
</head>
<body>
    <div class="navbar">
        <a href="/">Ana Sayfa</a>
        <a href="/dashboard">Kontrol Paneli</a>
        <a href="/patients">Hasta Yönetimi</a>
    </div>

    <div class="container">
        <h1>Hasta Geçmişi</h1>

        <h2>{{ patient[0] }} (ID: {{ patient[1] }})</h2>
        <p>E-posta: {{ patient[2] }}</p>
        <p>Telefon: {{ patient[3] }}</p>

        <h2>Sınıflandırma Geçmişi</h2>
        {% if history %}
            <table>
                <thead>
                    <tr>
                        <th>Dosya Adı</th>
                        <th>Sonuç</th>
                        <th>Yükleme Tarihi</th>
                        <th>Önizleme</th>
                    </tr>
                </thead>
                <tbody>
                    {% for record in history %}
                    <tr>
                        <td>{{ record[0] }}</td>
                        <td>{{ record[1] }}</td>
                        <td>{{ record[2] }}</td>
                        <td>
                            <img src="/static/uploads/{{ record[0] }}" alt="Preview" style="max-width: 100px; max-height: 100px;">
                        </td>
                    </tr>
                    {% endfor %}
                </tbody>
            </table>
        {% else %}
            <p>Bu hasta için sınıflandırma geçmişi bulunamadı.</p>
        {% endif %}
    </div>
</body>
</html>


Writing templates/patient_history.html


In [ ]:
%%writefile templates/upload.html
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Dosya Yükleme</title>
    <link rel="stylesheet" href="/static/style.css">
</head>
<body>

    <div class="navbar">
        <a href="/">Ana Sayfa</a>
        <a href="/dashboard">Kontrol Paneli</a>
        <a href="/patients">Hasta Yönetimi</a>
        <a href="/upload" class="active">Dosya Yükleme</a>
    </div>

    <div class="container">
        <h1>Dosya Yükleme</h1>
        <p>Buradan dosyanızı yükleyin ve sınıflandırma sonucunu görün.</p>

        {% if error %}
            <p class="error">{{ error }}</p>
        {% endif %}

        <form id="upload-form" method="POST" enctype="multipart/form-data">

            <label for="patient_id">Hastanın Kimlik Numarası Giriniz:</label>
            <input type="text" name="patient_id" id="patient_id" placeholder="Hasta ID'sini girin" required>

            <label for="file-input">Görüntü Yükleyin:</label>
            <input type="file" name="file" id="file-input" accept="image/*" required>

            <button type="submit">Dosya Yükle</button>
        </form>

        {% if label %}
            <div class="result">
                <h2>Sınıflandırma Sonucu</h2>
                <p><strong>{{ label }}</strong></p>
            </div>
        {% endif %}

        {% if image_url %}
            <div class="image-preview">
                <h2>Yüklenen Görüntü</h2>
                <img src="{{ image_url }}" alt="Yüklenen Görüntü" style="max-width: 100%; border: 1px solid #ddd; padding: 10px;">
            </div>
        {% endif %}
    </div>

    <footer>
        <p>&copy; 2025 Çocuklar İçin Diş Radyografisi Sınıflandırıcısı | Developed by Walid Mohmmed</p>
    </footer>

</body>
</html>


Writing templates/upload.html


In [ ]:
%%writefile templates/patients.html
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Hasta Yönetimi</title>
    <link rel="stylesheet" href="/static/style.css">
</head>
<body>


    <div class="navbar">
        <a href="/">Ana Sayfa</a>
        <a href="/dashboard">Kontrol Paneli</a>
        <a href="/patients" class="active">Hasta Yönetimi</a>
        <a href="/upload">Dosya Yükleme</a>
    </div>

    <div class="container">
        <h1>Hasta Yönetimi</h1>
        <p>Mevcut hasta kayıtlarını yönetin veya yeni hasta ekleyin.</p>


        {% if error %}
        <p class="error">{{ error }}</p>
        {% endif %}

        <h2>Yeni Hasta Ekle</h2>
        <form method="POST" action="/patients">
            <input type="text" name="name" placeholder="Hasta Adı" required>
            <input type="text" name="patient_id" placeholder="Hasta Kimliği" required>
            <input type="email" name="email" placeholder="E-posta">
            <input type="text" name="phone" placeholder="Telefon">
            <button type="submit">Yeni Hasta Ekle</button>
        </form>


        <h2>Hasta Listesi</h2>
        {% if patients %}
        <table>
            <thead>
                <tr>
                    <th>Ad</th>
                    <th>Hasta Kimliği</th>
                    <th>E-posta</th>
                    <th>Telefon</th>
                    <th>Geçmiş</th>
                </tr>
            </thead>
            <tbody>
                {% for patient in patients %}
                <tr>
                    <td>{{ patient[1] }}</td>
                    <td>{{ patient[2] }}</td>
                    <td>{{ patient[3] }}</td>
                    <td>{{ patient[4] }}</td>
                    <td><a href="/patient/{{ patient[0] }}" class="view-history-link">Geçmişi Gör</a></td>
                </tr>
                {% endfor %}
            </tbody>
        </table>
        {% else %}
        <p>Şu anda herhangi bir hasta kaydı bulunmamaktadır.</p>
        {% endif %}
    </div>


    <footer>
        <p>&copy; 2025 Çocuklar İçin Diş Radyografisi Sınıflandırıcısı | Developed by Walid Mohmmed</p>
    </footer>

</body>
</html>



Writing templates/patients.html


In [ ]:
%%writefile templates/dashboard.html
<!DOCTYPE html>
<html lang="tr">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Kontrol Paneli - Sınıflandırma Geçmişi</title>
    <link rel="stylesheet" href="/static/style.css">
    <script src="https://cdn.jsdelivr.net/npm/chart.js"></script>
</head>
<body>

    <!-- Navbar -->
    <div class="navbar">
        <a href="/">Ana Sayfa</a>
        <a href="/dashboard" class="active">Kontrol Paneli</a>
        <a href="/patients">Hasta Yönetimi</a>
        <a href="/upload">Dosya Yükleme</a>
    </div>


    <div class="container">
        <h1>Kontrol Paneli</h1>
        <p>Yüklenen görüntülerin sınıflandırma geçmişini görüntüleyin ve yönetim araçlarına erişin.</p>


        <h2>Yüklenen Görüntülerin Geçmişi</h2>
        {% if rows %}
        <table>
            <thead>
                <tr>
                    <th>Dosya Adı</th>
                    <th>Sonuç</th>
                    <th>Yükleme Tarihi</th>
                    <th>Hasta Adı</th>
                </tr>
            </thead>
            <tbody>
                {% for row in rows %}
                <tr>
                    <td>{{ row[0] }}</td>
                    <td>{{ row[1] }}</td>
                    <td>{{ row[2] }}</td>
                    <td>{{ row[3] or "Bilinmiyor" }}</td>
                </tr>
                {% endfor %}
            </tbody>
        </table>
        {% else %}
        <p>Şu anda herhangi bir sınıflandırma geçmişi bulunmamaktadır.</p>
        {% endif %}


        <div class="download-section">
            <h2>Geçmişi İndir</h2>
            <a href="/download/excel" class="download-button excel-download">Excel Formatında İndir</a>
        </div>
    </div>

    <footer>
        <p>&copy; 2025 Çocuklar İçin Diş Radyografisi Sınıflandırıcısı | Developed by Walid Mohmmed</p>
    </footer>

</body>
</html>


    <script>

        fetch('/stats')
            .then(response => response.json())
            .then(data => {
                document.getElementById('chart-spinner').style.display = 'none';
                document.getElementById('classificationChart').style.display = 'block';
                const ctx = document.getElementById('classificationChart').getContext('2d');
                new Chart(ctx, {
                    type: 'doughnut',
                    data: {
                        labels: ['Sağlıklı', 'Çürük'],
                        datasets: [{
                            label: 'Sınıflandırma İstatistikleri',
                            data: [data.healthy, data.caries],
                            backgroundColor: [
                                'rgba(75, 192, 192, 0.6)',
                                'rgba(255, 99, 132, 0.6)'
                            ],
                            borderColor: [
                                'rgba(75, 192, 192, 1)',
                                'rgba(255, 99, 132, 1)'
                            ],
                            borderWidth: 1
                        }]
                    },
                    options: {
                        responsive: true,
                        plugins: {
                            legend: {
                                position: 'top',
                            },
                            title: {
                                display: true,
                                text: 'Sınıflandırma Özeti'
                            },
                            tooltip: {
                                callbacks: {
                                    label: function(tooltipItem) {
                                        const dataset = tooltipItem.dataset;
                                        const total = dataset.data.reduce((prev, curr) => prev + curr, 0);
                                        const value = dataset.data[tooltipItem.dataIndex];
                                        const percentage = ((value / total) * 100).toFixed(2);
                                        return `${dataset.labels[tooltipItem.dataIndex]}: ${value} (${percentage}%)`;
                                    }
                                }
                            }
                        }
                    }
                });
            })
            .catch(error => {
                document.getElementById('chart-spinner').style.display = 'none';
                console.error('Grafik verileri alınırken hata oluştu:', error);
            });
    </script>
</body>
</html>


Writing templates/dashboard.html


In [ ]:
%%writefile templates/signup.html
<!DOCTYPE html>
<html lang="tr">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Kayıt Ol</title>
    <link rel="stylesheet" href="/static/style.css">
</head>
<body>

    <!-- Form Konteyneri -->
    <div class="form-container">
        <h2>Hesap Oluştur</h2>
        {% if error %}
            <p class="error">{{ error }}</p>
        {% endif %}
        <form method="POST" action="/signup">
            <input type="text" name="username" placeholder="Kullanıcı Adı" required>
            <input type="email" name="email" placeholder="E-posta" required>
            <input type="password" name="password" placeholder="Şifre" required>
            <button type="submit">Kayıt Ol</button>
        </form>
        <p class="form-footer">Zaten bir hesabınız var mı? <a href="/login">Giriş Yapın</a>.</p>
    </div>
</body>
</html>


Writing templates/signup.html


In [ ]:
%%writefile templates/login.html
<!DOCTYPE html>
<html lang="tr">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Giriş Yap</title>
    <link rel="stylesheet" href="/static/style.css">
</head>
<body>


    <div class="form-container">
        <h2>Tekrar Hoş Geldiniz</h2>
        {% if error %}
            <p class="error">{{ error }}</p>
        {% endif %}
        <form method="POST" action="/login">
            <input type="email" name="email" placeholder="E-posta" required>
            <input type="password" name="password" placeholder="Şifre" required>
            <button type="submit">Giriş Yap</button>
        </form>
        <p class="form-footer">Hesabınız yok mu? <a href="/signup">Buradan kayıt olun</a>.</p>
    </div>
</body>
</html>


Writing templates/login.html


In [ ]:
mkdir static


mkdir: cannot create directory ‘static’: File exists


In [ ]:
%%writefile static/style.css
body {
    font-family: Arial, sans-serif;
    background-color: #f4f4f9;
    margin: 0;
    padding: 0;
}

.navbar {
    background-color: #007bff;
    color: #fff;
    padding: 15px;
    text-align: center;
    font-size: 18px;
    font-weight: bold;
    box-shadow: 0 2px 5px rgba(0, 0, 0, 0.1);
    position: fixed;
    top: 0;
    width: 100%;
    z-index: 1000;
    display: flex;
    justify-content: center;
    gap: 15px;
}

.navbar a {
    color: #fff;
    text-decoration: none;
    padding: 10px 15px;
    border-radius: 5px;
    transition: all 0.3s ease;
    font-size: 16px;
}

.navbar a:hover {
    background-color: #0056b3;
    color: #f4f4f9;
    transform: scale(1.05);
}

/* Dashboard table styling */
table {
    width: 100%;
    border-collapse: collapse;
    margin-top: 20px;
}

table th, table td {
    border: 1px solid #ddd;
    padding: 10px;
    text-align: center;
    font-size: 14px;
}

table th {
    background-color: #007bff;
    color: #fff;
    font-weight: bold;
    font-size: 16px;
}

table tr:nth-child(even) {
    background-color: #f9f9f9;
}

table tr:hover {
    background-color: #f1f1f1;
}


#classificationChart {
    margin: 20px auto;
    max-width: 80%;
    padding: 20px;
    background: #fff;
    border-radius: 10px;
    box-shadow: 0 4px 6px rgba(0, 0, 0, 0.1);
}


.download-button {
    display: inline-block;
    background-color: #007bff;
    color: #fff;
    padding: 10px 20px;
    margin-top: 20px;
    border-radius: 5px;
    text-decoration: none;
    font-size: 16px;
    font-weight: bold;
    transition: all 0.3s ease;
}

.download-button:hover {
    background-color: #0056b3;
    transform: scale(1.05);
}


.form-container {
    max-width: 400px;
    margin: 120px auto;
    background: #fff;
    padding: 20px;
    border-radius: 10px;
    box-shadow: 0 4px 10px rgba(0, 0, 0, 0.1);
    text-align: center;
}

.form-container h2 {
    color: #333;
    margin-bottom: 20px;
    font-size: 24px;
}

.form-container .error {
    color: red;
    font-size: 14px;
    margin-bottom: 15px;
    text-align: center;
}


.form-container input[type="text"],
.form-container input[type="email"],
.form-container input[type="password"] {
    width: 90%;
    padding: 10px;
    margin: 10px 0;
    border: 1px solid #ccc;
    border-radius: 5px;
    font-size: 16px;
    box-shadow: 0 1px 3px rgba(0, 0, 0, 0.1);
    transition: all 0.2s ease-in-out;
}

.form-container input[type="text"]:focus,
.form-container input[type="email"]:focus,
.form-container input[type="password"]:focus {
    outline: none;
    border-color: #007bff;
    box-shadow: 0 0 8px rgba(0, 123, 255, 0.5);
}


.form-container button {
    background-color: #007bff;
    color: #fff;
    border: none;
    padding: 10px 20px;
    border-radius: 5px;
    font-size: 16px;
    cursor: pointer;
    transition: all 0.3s;
}

.form-container button:hover {
    background-color: #0056b3;
    transform: scale(1.05);
}


.form-container .form-footer {
    margin-top: 15px;
    font-size: 14px;
}

.form-container .form-footer a {
    color: #007bff;
    text-decoration: none;
    font-weight: bold;
}

.form-container .form-footer a:hover {
    text-decoration: underline;
}


.error {
    color: red;
    font-size: 14px;
    margin-bottom: 15px;
    text-align: center;
}


.container {
    max-width: 800px;
    margin: 100px auto 50px auto;
    background: #fff;
    padding: 20px;
    border-radius: 8px;
    box-shadow: 0 4px 6px rgba(0, 0, 0, 0.1);
    text-align: center;
}

/* Hero Section */
.container h1 {
    color: #333;
    font-size: 28px;
    margin-bottom: 10px;
}

.container p {
    color: #666;
    font-size: 16px;
    margin-bottom: 20px;
}

/* Input styling */
input[type='file'] {
    display: block;
    margin: 20px auto;
    padding: 5px;
    border: 1px solid #ccc;
    border-radius: 5px;
    font-size: 16px;
    cursor: pointer;
    max-width: 90%;
}


.result {
    margin-top: 20px;
    font-size: 20px;
    color: #007bff;
    font-weight: bold;
}


.image-preview img {
    max-width: 100%;
    margin-top: 20px;
    border: 1px solid #ccc;
    border-radius: 5px;
    box-shadow: 0 4px 6px rgba(0, 0, 0, 0.1);
}


.about-section {
    background-color: #fff;
    padding: 20px;
    margin: 30px auto;
    border-radius: 8px;
    box-shadow: 0 4px 6px rgba(0, 0, 0, 0.1);
    text-align: center;
}

.about-section h2 {
    color: #333;
    margin-bottom: 15px;
}

.about-section p {
    color: #555;
    font-size: 14px;
    line-height: 1.6;
}


footer {
    background-color: #f4f4f9;
    color: #666;
    text-align: center;
    padding: 10px 0;
    margin-top: 20px;
    font-size: 14px;
    border-top: 1px solid #ddd;
}




.download-button {
    display: inline-block;
    margin: 10px auto;
    padding: 12px 20px;
    background-color: #007bff; -
    color: #fff; -
    font-size: 16px;
    font-weight: bold;
    text-align: center;
    text-decoration: none;
    border-radius: 8px;
    transition: all 0.3s ease;
    border: 2px solid #0056b3;
    cursor: pointer;
}

.download-button:hover {
    background-color: #0056b3;
    border-color: #004080;
    transform: scale(1.05);
}


.excel-download {
    background-color: #28a745;
    border-color: #1c7430;
}

.excel-download:hover {
    background-color: #1c7430;
    border-color: #155724;
    transform: scale(1.05);
}


button {
    background: linear-gradient(45deg, #007bff, #0056b3);
    color: #fff;
    padding: 12px 25px;
    border: none;
    border-radius: 25px;
    font-size: 16px;
    font-weight: bold;
    cursor: pointer;
    display: inline-flex;
    align-items: center;
    justify-content: center;
    gap: 10px;
    transition: all 0.3s ease-in-out;
    box-shadow: 0 4px 8px rgba(0, 0, 0, 0.1);
}

button:hover {
    background: linear-gradient(45deg, #0056b3, #007bff);
    box-shadow: 0 6px 12px rgba(0, 0, 0, 0.2);
    transform: scale(1.05);
}

button:active {
    transform: scale(0.95);
    box-shadow: 0 3px 6px rgba(0, 0, 0, 0.2);
}

button i {
    font-size: 18px;
}


.navbar .logout {
    color: #ff4d4d;
    font-weight: bold;
    text-decoration: none;
    padding: 10px 15px;
    border-radius: 5px;
    transition: all 0.3s ease;
}

.navbar .logout:hover {
    background-color: #ff6666;
    color: white;
    transform: scale(1.05);
}



Writing static/style.css
